# **Data Preparation**


In [80]:
import numpy as np
import pandas as pd


In [81]:
#File Path
file_path = "../data/raw/online retail.xlsx"
csv_output_path = "../data/interim/online_retail.csv"

#Read Excel File
df_raw= pd.read_excel(file_path, engine="openpyxl")

# Save as CSV
df_raw.to_csv(csv_output_path, index=False)

# **Data Collection**


In [82]:
# Read CSV file
df = pd.read_csv('../data/interim/online_retail.csv')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [83]:
#Check the data shape
df.shape

(541909, 8)

This means our dataset contains:
- 541,909 rows → Each row represents a transaction record.
- 8 columns → These are the different attributes describing each transaction.
This gives us an initial understanding of the dataset's size before diving deeper.

# **Data Cleaning**


##### **Checking Datatype**

In [84]:
#Check the Basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [85]:
#Converting CustomerID Data Type
df['CustomerID'] = df['CustomerID'].astype('object')

In [86]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

In the dataset, CustomerID was initially stored as a numeric type but was converted to object because it represents a categorical identifier rather than a numerical value

The dataset contains **541,909 rows and 8 columns**. Five columns (`InvoiceNo`, `StockCode`, `Description`, `InvoiceDate`, `Country`) are `object` types, while `Quantity` is an integer and `UnitPrice` & `CustomerID` are floats.  Missing values appear in `Description` and `CustomerID`, likely indicating incomplete product details and guest transactions.



##### **Checking Missing Value**
In this step, we checked for missing values in the dataset using the `.isnull().sum()` method. The results show the number of missing values for each column in the dataset.

In [87]:
# Count missing values per column
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

- **CustomerID** has a significant number of missing values, with **135,080** missing entries.
- **Description** also has some missing values, totaling **1,454** entries.
- Other columns, such as **InvoiceNo, StockCode, Quantity, InvoiceDate, UnitPrice, Country, and Is_Canceled**, have no missing values (0 missing values).



In [88]:
missing_percentage = df['CustomerID'].isnull().mean() * 100
print(f'CustomerID Missing Value: {df["CustomerID"].isnull().sum()}')
print(f'Missing Value Percentage: {missing_percentage:.2f} %')

CustomerID Missing Value: 135080
Missing Value Percentage: 24.93 %


In [89]:
#Calculate Missing Value  Proportionnn by country
missing_by_country = df.groupby('Country')['CustomerID'].apply(lambda x: x.isna().mean() * 100).sort_values(ascending=False)
missing_by_country = missing_by_country[missing_by_country > 0].round(2)

# Convert to DataFrame and rename column
missing_by_country = missing_by_country.to_frame(name="Missing Value Proportion")

# Print properly formatted output
print(missing_by_country)

                Missing Value Proportion
Country                                 
Hong Kong                         100.00
Unspecified                        45.29
United Kingdom                     26.96
Israel                             15.82
Bahrain                            10.53
EIRE                                8.67
Switzerland                         6.24
Portugal                            2.57
France                              0.77


Hong Kong has the highest proportion of missing CustomerID values at 100%, suggesting either a different data recording system or unregistered customers. The United Kingdom (26.96%) and Unspecified (45.29%) categories also show significant missing values, which might be due to guest checkouts or incomplete records, while other countries like Israel (15.82%), Bahrain (10.53%), and EIRE (8.67%) have moderate levels of missing data. To address this, further investigation is needed, particularly for Hong Kong and the Unspecified category, to determine whether the missing values should be removed, imputed, or retained based on business relevance.

In [90]:
#Hongkong CustomerID missing value
hongkong_missing = missing_by_country.loc["Hong Kong", "Missing Value Proportion"]

total_missing = missing_by_country["Missing Value Proportion"].sum()


percentage_hongkong = (hongkong_missing / total_missing) * 100
print(f"Hong Kong contributes {percentage_hongkong:.2f}% of all missing CustomerID values.")


Hong Kong contributes 46.11% of all missing CustomerID values.


In [91]:
#CustomerID missing value by InvoiceNo
num_missing_invoices = df[df['CustomerID'].isna()]['InvoiceNo'].nunique()
print(f'Mising CustomerID by  Invoices:  {num_missing_invoices}')
InvoiceNo_unique = df['InvoiceNo'].nunique()

print(f'Amount Unique InvoiceNo: ',InvoiceNo_unique)

Mising CustomerID by  Invoices:  3710
Amount Unique InvoiceNo:  25900


This suggests that these transactions were recorded without an associated customer identifier, which may impact customer-level analysis.


In [92]:
#CustomerID  missing value by canceled transaction
missing_returns = df[df['CustomerID'].isna() & (df['Quantity'] < 0)]
print(f"Total missing CustomerIDs in canceled transactions: {missing_returns.shape[0]}")


Total missing CustomerIDs in canceled transactions: 1719


The fact that **1,719 CustomerIDs are missing in canceled transactions** suggests that many orders without an assigned customer were later canceled. This could indicate that guest checkouts, system errors, or unregistered customers are more prone to cancellations. Understanding this pattern can help in improving customer retention strategies and identifying potential data quality issues.

Due to significant missing values in CustomerID, customer-based analysis may lead to biased insights. Thus, we decided to exclude it and focus on more reliable features.

##### **Checking Duplication**

In [93]:
# Identify duplicate rows (excluding the first occurrence)
df.duplicated().sum()

5268

In [94]:
# Remove duplicate rows, keeping only the first occurrence
df.drop_duplicates(keep='first', inplace=True)

In [95]:
#  Count the number of duplicated rows again to verify removal
df.duplicated().sum()

0

##### **Checking Irrelevant Value**

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536641 non-null  object        
 1   StockCode    536641 non-null  object        
 2   Description  535187 non-null  object        
 3   Quantity     536641 non-null  int64         
 4   InvoiceDate  536641 non-null  datetime64[ns]
 5   UnitPrice    536641 non-null  float64       
 6   CustomerID   401604 non-null  object        
 7   Country      536641 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 36.8+ MB


##### **Checking Irrelevant Value : InvoiceNo**

Since `InvoiceNo` is expected to have a numeric format, we need to check if there are any values with a different format. The code below identifies non-numeric `InvoiceNo`, which may indicate anomalies or specific cases that require further investigation. 

In [97]:
#Checking InvoiceNo format
invoice_different_format =  df[~df['InvoiceNo'].str.isnumeric()]

invoice_different_format['InvoiceNo']

141       C536379
154       C536383
235       C536391
236       C536391
237       C536391
           ...   
540449    C581490
541541    C581499
541715    C581568
541716    C581569
541717    C581569
Name: InvoiceNo, Length: 9254, dtype: object

In [98]:
# Counts Unique non-numeric InvoiceNo
invoice_different_format['InvoiceNo'].nunique()

3839

In [99]:
# Group by InvoiceNo and sum the Quantity column
invoice_summary = invoice_different_format[['InvoiceNo', 'Quantity']]
print(invoice_summary)

       InvoiceNo  Quantity
141      C536379        -1
154      C536383        -1
235      C536391       -12
236      C536391       -24
237      C536391       -24
...          ...       ...
540449   C581490       -11
541541   C581499        -1
541715   C581568        -5
541716   C581569        -1
541717   C581569        -5

[9254 rows x 2 columns]


In [100]:
# Filter invoices where Quantity is less than 0
negative_invoice_summary = invoice_different_format[invoice_different_format['Quantity'] < 0][['InvoiceNo', 'Quantity']]

# Display the result
print(negative_invoice_summary)


       InvoiceNo  Quantity
141      C536379        -1
154      C536383        -1
235      C536391       -12
236      C536391       -24
237      C536391       -24
...          ...       ...
540449   C581490       -11
541541   C581499        -1
541715   C581568        -5
541716   C581569        -1
541717   C581569        -5

[9251 rows x 2 columns]


In [101]:
# Filter invoices where Quantity is greater than or equal to 0
positive_invoice_summary = invoice_different_format[invoice_different_format['Quantity'] >= 0][['InvoiceNo', 'Quantity']]

# Display the result
print(positive_invoice_summary)


       InvoiceNo  Quantity
299982   A563185         1
299983   A563186         1
299984   A563187         1


Non-numeric `InvoiceNo` values (9,254 entries, 3,839 unique) start with 'C' and 'A'. Specifically, invoices starting with 'C' contain negative `Quantity` values, totaling 9,251 entries, suggesting they may represent returns or adjustments. In contrast, invoices starting with 'A' have positive `Quantity` values but account for only 3 entries, indicating a rare but structured pattern. This structured distinction implies an intentional data classification rather than an anomaly or human error. Further clarification from the team is needed to understand the context behind these invoice formats and ensure accurate data handling.

##### **Checking Irrelevant Value : Quantity**

In [102]:
df['Quantity'].describe()

count    536641.000000
mean          9.620029
std         219.130156
min      -80995.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       80995.000000
Name: Quantity, dtype: float64

In [103]:
df.loc[df['Quantity']<=0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


There are negative values in the **Quantity** column, and as previously explained, most of them have an **InvoiceNo** that starts with the letter **'C'**. This could indicate product returns (refunds) or order cancellations. However, this needs further investigation, and since there is no definitive explanation at this stage, I will replace the negative values with **0**, as negative Quantity values do not make sense without further clarification.

In [104]:
# Replace negative values in the Quantity column with 0,
df.loc[df['Quantity']<0, 'Quantity'] = 0 

In [105]:
# Check the statistical summary of the Quantity column  
# to verify the changes after replacing negative values with 0
df['Quantity'].describe()

count    536641.000000
mean         10.519172
std         156.036720
min           0.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       80995.000000
Name: Quantity, dtype: float64

##### **Checking Irrelevant Value : StockCode**

In [106]:
# Check the number of unique StockCode values
df['StockCode'].nunique()

4070

In [107]:
df['StockCode'].unique()[:20]

array(['85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730',
       '22633', '22632', '84879', '22745', '22748', '22749', '22310',
       '84969', '22623', '22622', '21754', '21755', '21777'], dtype=object)

In [108]:
average_length = df['StockCode'].apply(len).mean()
print(average_length)

5.087033976159108


There are 4,070 unique `StockCode` values in the dataset, with an average length of approximately 5.08 characters. This suggests that most stock codes follow a structured format, typically consisting of 5 or 6 characters, which may include both numbers and letters.  

To further analyze this, I will check `StockCode` values with lengths below and above 5 to identify any irregularities or patterns.

In [109]:
# Check the StockCode with len less than 5
df[df['StockCode'].apply(len) < 5]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.00,12583.0,France
141,C536379,D,Discount,0,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
386,536403,POST,POSTAGE,1,2010-12-01 11:27:00,15.00,12791.0,Netherlands
1123,536527,POST,POSTAGE,1,2010-12-01 13:04:00,18.00,12662.0,Germany
1423,536540,C2,CARRIAGE,1,2010-12-01 14:05:00,50.00,14911.0,EIRE
...,...,...,...,...,...,...,...,...
541540,581498,DOT,DOTCOM POSTAGE,1,2011-12-09 10:26:00,1714.17,NaN,United Kingdom
541541,C581499,M,Manual,0,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541730,581570,POST,POSTAGE,1,2011-12-09 11:59:00,18.00,12662.0,Germany
541767,581574,POST,POSTAGE,2,2011-12-09 12:09:00,18.00,12526.0,Germany


In [110]:
# Make a list of unique StockCode with len less than 5
stockcode_list = list(df.loc[df['StockCode'].apply(len) < 5, 'StockCode'].unique())
stockcode_list


['POST', 'D', 'C2', 'DOT', 'M', 'S', 'm', 'PADS', 'B', 'CRUK']

The analysis reveals that several `StockCode` values have a length of less than 5 characters. These include short, non-numeric codes such as `'POST'`, `'D'`, `'C2'`, `'DOT'`, `'M'`, `'S'`, `'m'`, `'PADS'`, `'B'`, and `'CRUK'`. These codes differ significantly from the typical 5-6 character format observed in most stock codes, suggesting that they may represent special transactions, discounts, or non-product-related entries rather than standard inventory items. Further investigation is needed to determine their exact meaning and impact on the dataset.

In [111]:
# Count the number of rows where 'StockCode' matches to stockdoe_list
count = df[df['StockCode'].isin(stockcode_list)].shape[0]
print(count)

2839


In [112]:
# Filtering the dataset to show the first 3 rows where StockCode is "POST"
df[df['StockCode']== "POST"].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.0,12583.0,France
386,536403,POST,POSTAGE,1,2010-12-01 11:27:00,15.0,12791.0,Netherlands
1123,536527,POST,POSTAGE,1,2010-12-01 13:04:00,18.0,12662.0,Germany


In [113]:
# Filtering the dataset to show the first 3 rows where StockCode is "D"
df[df['StockCode']== 'D'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,0,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
9038,C537164,D,Discount,0,2010-12-05 13:21:00,29.29,14527.0,United Kingdom
14498,C537597,D,Discount,0,2010-12-07 12:34:00,281.00,15498.0,United Kingdom


In [114]:
# Filtering the dataset to show the first 3 rows where StockCode is "C2"
df[df['StockCode']== 'C2'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1423,536540,C2,CARRIAGE,1,2010-12-01 14:05:00,50.0,14911.0,EIRE
12119,537368,C2,CARRIAGE,1,2010-12-06 12:40:00,50.0,14911.0,EIRE
12452,537378,C2,CARRIAGE,1,2010-12-06 13:06:00,50.0,14911.0,EIRE


In [115]:
# Filtering the dataset to show the first 3 rows where StockCode is "DOT"
df[df['StockCode']== 'DOT'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1814,536544,DOT,DOTCOM POSTAGE,1,2010-12-01 14:32:00,569.77,NaN,United Kingdom
3041,536592,DOT,DOTCOM POSTAGE,1,2010-12-01 17:06:00,607.49,NaN,United Kingdom
5450,536862,DOT,DOTCOM POSTAGE,1,2010-12-03 11:13:00,254.43,NaN,United Kingdom


In [116]:
# Filtering the dataset to show the first 3 rows where StockCode is "M"
df[df['StockCode']== 'M'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274.0,United Kingdom
2250,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274.0,United Kingdom
5684,536865,M,Manual,1,2010-12-03 11:28:00,2.55,NaN,United Kingdom


In [117]:
# Filtering the dataset to show the first 3 rows where StockCode is "m"
df[df['StockCode']== 'm'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
40383,539736,m,Manual,1,2010-12-21 15:18:00,2.55,NaN,United Kingdom


In [118]:
# Filtering the dataset to show the first 3 rows where StockCode is "S"
df[df['StockCode']== 'S'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14436,C537581,S,SAMPLES,0,2010-12-07 12:03:00,12.95,NaN,United Kingdom
14437,C537581,S,SAMPLES,0,2010-12-07 12:03:00,52.00,NaN,United Kingdom
96680,C544580,S,SAMPLES,0,2011-02-21 14:25:00,5.74,NaN,United Kingdom


In [119]:
# Filtering the dataset to show the first 3 rows where StockCode is "S"
df[df['StockCode']== 'PADS'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
157195,550193,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-04-15 09:27:00,0.001,13952.0,United Kingdom
279045,561226,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-07-26 10:13:00,0.001,15618.0,United Kingdom
358655,568158,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-09-25 12:22:00,0.000,16133.0,United Kingdom


In [120]:
# Filtering the dataset to show the first 3 rows where StockCode is "B"
df[df['StockCode']== 'B'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [121]:
# Filtering the dataset to show the first 3 rows where StockCode is "CRUK"
df[df['StockCode']== 'CRUK'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
317508,C564763,CRUK,CRUK Commission,0,2011-08-30 10:49:00,1.60,14096.0,United Kingdom
324023,C565382,CRUK,CRUK Commission,0,2011-09-02 15:45:00,13.01,14096.0,United Kingdom
333779,C566216,CRUK,CRUK Commission,0,2011-09-09 15:17:00,15.96,14096.0,United Kingdom


After analyzing the stock codes ['POST', 'D', 'C2', 'DOT', 'M', 'S', 'm', 'PADS', 'B', 'CRUK'], I found they consistently refer to non-product items like postage fees, discounts, carriage costs, manual adjustments, and commissions. Since these entries—totaling only 2,839 out of 541,909 (≈0.52%)—do not represent actual sales, I decided to drop them.

In [122]:
# Drop rows where 'StockCode' is in stockcode_list to remove non-product entries
df = df.drop(df[df['StockCode'].isin(stockcode_list)].index)


In [123]:
# Filter the dataset to check which rows contain stock codes from stockcode_list  
df[df['StockCode'].isin(stockcode_list)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [124]:
# Retrieve a unique list of 'StockCode' values that have more than 6 characters  
print(list(df.loc[df['StockCode'].apply(len) > 6, 'StockCode'].unique()))

['15056BL', 'BANK CHARGES', '15056bl', 'AMAZONFEE', 'DCGS0076', 'DCGS0003', 'gift_0001_40', 'DCGS0070', 'gift_0001_50', 'gift_0001_30', 'gift_0001_20', 'DCGS0055', 'DCGS0072', 'DCGS0074', 'DCGS0069', 'DCGS0057', 'DCGSSBOY', 'DCGSSGIRL', 'gift_0001_10', 'DCGS0004', 'DCGS0073', 'DCGS0071', 'DCGS0068', 'DCGS0067', 'DCGS0066P', '79323GR', '79323LP']


I extracted stock codes with more than six characters and found a mix of financial transaction labels (e.g., 'BANK CHARGES', 'AMAZONFEE') and structured product codes (e.g., 'gift_0001_40', 'DCGS0076'). Some of these entries seem to represent charges or fees rather than actual products, so I will analyze them further to determine if they should be excluded from the dataset.

In [125]:
# Filtering the dataset to show the first 3 rows where StockCode is "BANK CHARGES"
df[df['StockCode']== 'BANK CHARGES'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
4406,536779,BANK CHARGES,Bank Charges,1,2010-12-02 15:08:00,15.00,15823.0,United Kingdom
14435,C537572,BANK CHARGES,Bank Charges,0,2010-12-07 12:00:00,95.38,NaN,United Kingdom
28992,C538680,BANK CHARGES,Bank Charges,0,2010-12-13 17:10:00,966.92,NaN,United Kingdom


In [126]:
# Filtering the dataset to show the first 3 rows where StockCode is "AMAZONFEE"
df[df['StockCode']=='AMAZONFEE'].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
14514,C537600,AMAZONFEE,AMAZON FEE,0,2010-12-07 12:41:00,1.00,NaN,United Kingdom
15016,C537630,AMAZONFEE,AMAZON FEE,0,2010-12-07 15:04:00,13541.33,NaN,United Kingdom
15017,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom


In [127]:
# Filtering the dataset to show the first 3 rows where StockCode start with "150"
df[df['StockCode'].str.startswith('150')].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
132,536381,15056BL,EDWARDIAN PARASOL BLACK,2,2010-12-01 09:41:00,5.95,15311.0,United Kingdom
133,536381,15056N,EDWARDIAN PARASOL NATURAL,2,2010-12-01 09:41:00,5.95,15311.0,United Kingdom
281,536396,15056BL,EDWARDIAN PARASOL BLACK,6,2010-12-01 10:51:00,4.95,17850.0,United Kingdom


In [128]:
# Filtering the dataset to show the first 3 rows where StockCode start with "DCG"
df[df['StockCode'].str.startswith('DCG')].head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
21326,538071,DCGS0076,SUNJAR LED NIGHT NIGHT LIGHT,1,2010-12-09 14:09:00,16.13,NaN,United Kingdom
24906,538349,DCGS0003,BOXED GLASS ASHTRAY,1,2010-12-10 14:59:00,2.51,NaN,United Kingdom
36460,539451,DCGS0003,BOXED GLASS ASHTRAY,1,2010-12-17 16:59:00,2.51,NaN,United Kingdom


In [129]:
# Filtering the dataset to show the first 3 rows where StockCode start with 'gift_'
df[df['StockCode'].str.startswith('gift_')].head(3)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
38248,539492,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,1,2010-12-20 10:14:00,34.04,NaN,United Kingdom
42057,539958,gift_0001_50,Dotcomgiftshop Gift Voucher £50.00,1,2010-12-23 13:26:00,42.55,NaN,United Kingdom
44725,540238,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1,2011-01-05 14:44:00,25.53,NaN,United Kingdom


The code extracts unique `StockCode` values longer than six characters, revealing a mix of financial transaction labels (`'BANK CHARGES'`, `'AMAZONFEE'`), structured codes (`'gift_0001_40'`), and potential product codes (`'DCGS0076'`). The irrelevant entries, including `'BANK CHARGES'`, `'AMAZONFEE'`, and stock codes starting with `'gift_'`, should be excluded as they do not represent actual products. Further validation is needed to ensure the remaining codes are relevant.

In [130]:
# Remove rows where 'StockCode' starts with 'gift_'
df = df[~df['StockCode'].str.startswith('gift_')]

In [131]:
# Remove rows where 'StockCode' is either 'BANK CHARGES' or 'AMAZONFEE'
df = df[~df['StockCode'].isin(['BANK CHARGES', 'AMAZONFEE'])]

In [132]:
# Check if any StockCode still starts with 'gift_'
df['StockCode'].str.startswith('gift_').any()

False

In [133]:
# Check if 'BANK CHARGES' or 'AMAZONFEE' still exist
df['StockCode'].isin(['BANK CHARGES', 'AMAZONFEE']).any()

False

##### **Checking Irrelevant Value : Description**

In [134]:
df['Description'].value_counts()

Description
WHITE HANGING HEART T-LIGHT HOLDER     2357
REGENCY CAKESTAND 3 TIER               2189
JUMBO BAG RED RETROSPOT                2156
PARTY BUNTING                          1720
LUNCH BAG RED RETROSPOT                1625
                                       ... 
for online retail orders                  1
add stock to allocate online orders       1
allocate stock for dotcom orders ta       1
PAINTED HEART WREATH WITH BELL            1
MARIE ANTOIENETT TRINKET BOX GOLD         1
Name: count, Length: 4206, dtype: int64

In [135]:
# Select descriptions with fewer than 7 characters
short_descriptions = df.loc[df['Description'].str.len() < 7, 'Description']
print(short_descriptions)


6391      amazon
6392      amazon
7313           ?
13217      check
13218      check
           ...  
535331     check
535333     check
535334     check
535335      lost
535336     check
Name: Description, Length: 297, dtype: object


In [136]:
# Select unique value of descriptions with fewer than 7 characters
unique_short_descriptions = df.loc[df['Description'].str.len() < 7, 'Description'].unique()
print(unique_short_descriptions)

['amazon' '?' 'check' 'faulty' 'Found' 'found' 'Dotcom' 'MIA' 'broken'
 '?lost' 'adjust' 'FOUND' 'dotcom' 'FBA' 'ebay' 'test' '??' 'Amazon'
 'mouldy' '20713' 'CHECK' 'check?' 'AMAZON' 'wet?' 'lost??' '???' 'wet'
 'lost']


In [137]:
# Keep only rows where 'Description' has more than 7 characters
df = df[df['Description'].str.len() > 7]

In [138]:
# Select rows where 'Description' has 7 or fewer characters
df_filtered = df[df['Description'].str.len() <= 7]

# Display the filtered DataFrame
df_filtered


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


The code extracts unique values from the 'Description' column where the text length is fewer than 7 characters. The result includes various words, abbreviations, numbers, and symbols, such as 'amazon', '??', 'MIA', and '20713'. Many of these values seem inconsistent or possibly erroneous, like isolated punctuation marks, numbers, or vague terms. Since these descriptions may not provide meaningful information for analysis, I decided to drop entries with descriptions shorter than 7 characters.

##### **Checking Irrelevant Value : Quantity**

In [139]:
df['Quantity'].describe()

count    531829.000000
mean         10.507919
std         156.267861
min           0.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       80995.000000
Name: Quantity, dtype: float64

In [140]:
# Filter transactions where Quantity is 0 or negative
df[df['Quantity'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,0,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,0,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,0,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,0,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,0,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540448,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,0,2011-12-09 09:57:00,1.95,14397.0,United Kingdom
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,0,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,0,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,0,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [141]:
# Remove transactions with Quantity <= 0 since they are illogical for sales
df = df[df['Quantity'] > 0]

In [142]:
# Verify that QUantity with value < 0 already removed
df[df['Quantity']<= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


I removed transactions where **Quantity = 0** because they are illogical for a sales dataset. A transaction should represent at least one item sold, so these records might be errors, placeholders, or canceled orders. Keeping them could distort total sales and revenue calculations.  

However, we should further clarify their meaning, as they could indicate cancellations, inventory adjustments, or system errors. Understanding their context helps avoid mistakenly dropping meaningful data. **Since there is not enough information at this point, I decided to remove them.**

##### **Checking Irrelevant Value : InvoiceDate**

In [143]:
df['InvoiceDate']

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 523000, dtype: datetime64[ns]

In [144]:
# Checking Missing Value (NaT) in InvoiceDate
df['InvoiceDate'].isna().sum()

0

In [145]:
# Summarizes date range
df['InvoiceDate'].describe()

count                           523000
mean     2011-07-04 15:06:17.498547200
min                2010-12-01 08:26:00
25%                2011-03-28 11:50:30
50%                2011-07-20 11:38:00
75%                2011-10-19 11:41:00
max                2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

The `InvoiceDate` column contains **523,000** records with no missing values. The transactions range from **December 1, 2010, with a mean date around **July 4, 2011**. The presence of **December 2010** data might indicate an anomaly, as most transactions are concentrated within 2011. Additionally, since the dataset only includes transactions up to **December 9, 2011**, analyzing December 2011 trends is limited, and any conclusions about that month should be made with caution.

##### **Checking Irrelevant Value : UnitPrice**

In [146]:
# Provides summary statistics for UnitPrice
df['UnitPrice'].describe()

count    523000.000000
mean          3.271099
std           4.392311
min           0.000000
25%           1.250000
50%           2.080000
75%           4.130000
max         649.500000
Name: UnitPrice, dtype: float64

In [147]:
# Count the number of rows where UnitPrice is 0
df.loc[df['UnitPrice']== 0].shape[0]

463

In [148]:
# Remove rows where UnitPrice is 0
df.drop(df[df['UnitPrice']==0].index, inplace= True)

In [149]:
# Count again to verify that all rows with UnitPrice 0 are removed
df.loc[df['UnitPrice'] == 0].shape[0]

0

In [150]:
# Display summary statistics for UnitPrice after cleaning
df['UnitPrice'].describe()

count    522537.000000
mean          3.273997
std           4.393177
min           0.040000
25%           1.250000
50%           2.080000
75%           4.130000
max         649.500000
Name: UnitPrice, dtype: float64

This process begins by counting how many rows have `UnitPrice` equal to zero. Since a unit price of zero is likely incorrect or represents missing data, these rows are removed using `.drop()`, modifying the DataFrame in place. After that, the code verifies the removal by counting the remaining zero-value rows. Finally, it displays summary statistics of `UnitPrice` to check the distribution after cleaning. However, further clarification is needed to understand whether these zero values represent actual free items, errors in data entry, or missing information.

##### **Checking Irrelevant Value : CustomerID**

In [151]:
df['CustomerID'].describe()

count     391150.0
unique      4334.0
top        17841.0
freq        7667.0
Name: CustomerID, dtype: float64

In [152]:
df['CustomerID'].isnull().sum()

131387

The analysis shows **131,387 missing CustomerID values** (~24% of the data). Since these may represent valid transactions, they will **not be removed**, as discussed in the *Checking Missing Values* section.

##### **Checking Irrelevant Value : Country**

In [153]:
# Display the unique country names in the dataset  
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [154]:
# Count the number of unique country names 
df['Country'].nunique()

38

In [155]:
# Count the occurrences of each country in the dataset  
df['Country'].value_counts()


Country
United Kingdom          478871
Germany                   8643
France                    8085
EIRE                      7768
Spain                     2417
Netherlands               2322
Belgium                   1935
Switzerland               1927
Portugal                  1455
Australia                 1180
Norway                    1048
Channel Islands            743
Italy                      741
Finland                    647
Cyprus                     601
Unspecified                442
Sweden                     427
Austria                    384
Denmark                    367
Poland                     325
Japan                      321
Israel                     292
Hong Kong                  275
Singapore                  215
Iceland                    182
USA                        179
Canada                     150
Greece                     142
Malta                      109
United Arab Emirates        67
European Community          57
RSA                         57


In [156]:
# Removing 'European Community' as it is not a country, ensuring consistency in geographical data
df = df[df['Country'] != 'European Community']

In [157]:
# Replacing 'RSA' with 'South Africa' to maintain standard country names
df['Country'] = df['Country'].replace({'RSA': 'South Africa'})


In [158]:
# Verify the uniqie Value
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'Malta', 'South Africa'], dtype=object)

- **European Community** is removed because it is not a specific country but a regional entity. Additionally, it only appears in 57 rows, making it a minor portion of the dataset.
- **RSA** is replaced with **South Africa** to ensure consistency in country names, as 'RSA' is an abbreviation that might be unclear.

# **Export Cleaned Data**


In [159]:
# Processed File Path
processed_path = "../data/processed/cleaned_online_retail.csv"

# Save as CSV
df.to_csv(processed_path, index=False)